In [12]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    Dense,
    Flatten,
    Multiply,
    UpSampling2D,
    Activation,
    Conv2D,
)
import tensorflow as tf

# Load a pre-trained segmentation model (e.g., DenseNet121)
segmentation_model = tf.keras.applications.DenseNet121(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3),
    pooling=None,  # No global pooling, maintain spatial dimensions
)

# Input layer
input_layer = Input(shape=(224, 224, 3))

# Segmentation step
segmentation_output = segmentation_model(input_layer)

# Apply sigmoid activation using a Keras Activation layer
segmentation_mask = Activation("sigmoid")(segmentation_output)

# Ensure segmentation mask has 3 channels to match input
segmentation_mask = Conv2D(3, (1, 1), activation="sigmoid")(segmentation_mask)

# Resize segmentation output to match input dimensions (224x224)
segmentation_mask = UpSampling2D(
    size=(224 // segmentation_mask.shape[1], 224 // segmentation_mask.shape[2])
)(segmentation_mask)

# Apply segmentation mask to input image using Keras Multiply layer
segmented_input = Multiply()([input_layer, segmentation_mask])

# Classification network (VGG19 as base model)
base_model = VGG19(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
x = base_model(segmented_input)
x = Flatten(name="flatten")(x)
x = Dense(4096, activation="relu", name="fc1")(x)
x = Dense(4096, activation="relu", name="fc2")(x)
x = Dense(36, activation="softmax", name="predictions")(x)

# Final model
model = Model(inputs=input_layer, outputs=x)

# Freeze VGG19 base model layers
for layer in base_model.layers:
    layer.trainable = False

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_22      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ densenet121         │ (None, 7, 7,      │  7,037,504 │ input_layer_22[0… │
│ (Functional)        │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_6        │ (None, 7, 7,      │          0 │ densenet121[0][0] │
│ (Activation)        │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 7, 7, 3)   │      3,075 │ activation_6[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_5     │ (None, 224, 224,  │          0 │ conv2d_1[0][0]    │
│ (UpSampling2D)      │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_5          │ (None, 224, 224,  │          0 │ input_layer_22[0… │
│ (Multiply)          │ 3)                │            │ up_sampling2d_5[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vgg19 (Functional)  │ (None, 7, 7, 512) │ 20,024,384 │ multiply_5[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 25088)     │          0 │ vgg19[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fc1 (Dense)         │ (None, 4096)      │ 102,764,5… │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fc2 (Dense)         │ (None, 4096)      │ 16,781,312 │ fc1[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ predictions (Dense) │ (None, 36)        │    147,492 │ fc2[0][0]         │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 146,758,311 (559.84 MB)

 Trainable params: 126,650,279 (483.13 MB)

 Non-trainable params: 20,108,032 (76.71 MB)